In [2]:
pip install pandas plotly scikit-learn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("datasets/climate-stress/climate_change_dataset.csv")

df.head()

,Year,Country,Avg Temperature (°C),CO2 Emissions (Tons/Capita),Sea Level Rise (mm),Rainfall (mm),Population,Renewable Energy (%),Extreme Weather Events,Forest Area (%)
0,2006,UK,8.9,9.3,3.1,1441,530911230,20.4,14,59.8
1,2019,USA,31.0,4.8,4.2,2407,107364344,49.2,8,31.0
2,2014,France,33.9,2.8,2.2,1241,441101758,33.3,9,35.5
3,2010,Argentina,5.9,1.8,3.2,1892,1069669579,23.7,7,17.7
4,2007,Germany,26.9,5.6,2.4,1743,124079175,12.5,4,17.4


In [5]:
df = df[(df["Year"] >= 2000) & (df["Year"] <= 2020)]
df.head()

,Year,Country,Avg Temperature (°C),CO2 Emissions (Tons/Capita),Sea Level Rise (mm),Rainfall (mm),Population,Renewable Energy (%),Extreme Weather Events,Forest Area (%)
0,2006,UK,8.9,9.3,3.1,1441,530911230,20.4,14,59.8
1,2019,USA,31.0,4.8,4.2,2407,107364344,49.2,8,31.0
2,2014,France,33.9,2.8,2.2,1241,441101758,33.3,9,35.5
3,2010,Argentina,5.9,1.8,3.2,1892,1069669579,23.7,7,17.7
4,2007,Germany,26.9,5.6,2.4,1743,124079175,12.5,4,17.4


In [6]:
stress_vars = [
    "Avg Temperature (°C)",
    "CO2 Emissions (Tons/Capita)",
    "Sea Level Rise (mm)",
    "Extreme Weather Events"
]

scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[stress_vars] = scaler.fit_transform(df[stress_vars])

df_scaled["Climate Stress Index"] = df_scaled[stress_vars].mean(axis=1)


country_stress = (
    df_scaled
    .groupby("Country", as_index=False)
    .agg({
        "Climate Stress Index": "mean",
        "Avg Temperature (°C)": "mean",
        "CO2 Emissions (Tons/Capita)": "mean"
    })
)

map_fig = px.choropleth(
    country_stress,
    locations="Country",
    locationmode="country names",
    color="Climate Stress Index",
    color_continuous_scale="YlOrRd",
    hover_data={
        "Climate Stress Index": ":.2f",
        "Avg Temperature (°C)": ":.1f",
        "CO2 Emissions (Tons/Capita)": ":.2f"
    },
    title="Global Climate Stress Index (2000–2020)"
)

map_fig.update_layout(
    geo=dict(showframe=False, showcoastlines=True),
    coloraxis_colorbar=dict(title="Climate Stress")
)

map_fig.show()


stress_trend = (
    df_scaled.groupby("Year", as_index=False)
    ["Climate Stress Index"]
    .mean()
)



country_stress.to_csv("climate_stress_by_country.csv", index=False)

/var/folders/hk/2cjgkzrx4l52_0x78gkf21380000gn/T/ipykernel_7307/387284921.py:25: DeprecationWarning: The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.
  map_fig = px.choropleth(


In [8]:
df = pd.read_csv("climate_stress_by_country.csv")

data = df[['Country', 'Climate Stress Index']]
data.to_csv("analyzed_climate_stress_by_country.csv", index=False)
data.head()

,Country,Climate Stress Index
0,Argentina,0.497051
1,Australia,0.478994
2,Brazil,0.517556
3,Canada,0.510957
4,China,0.501540
